In [1]:
import pandas as pd

# Load the dataset and preprocess
game_data = pd.read_csv("games_data.csv", index_col=0)
del game_data["mp.1"]
del game_data["mp_opp.1"]
del game_data["index_opp"]
game_data.sort_values("date", inplace=True)
game_data.reset_index(drop=True, inplace=True)


In [2]:
def append_result_col(team_stats):
    team_stats["next_result"] = team_stats["won"].shift(-1)
    return team_stats

game_data = game_data.groupby("team", group_keys=False).apply(append_result_col)

C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\3695187440.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_stats["next_result"] = team_stats["won"].shift(-1)
C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\3695187440.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_stats["next_result"] = team_stats["won"].shift(-1)
C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\3695187440.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [3]:
game_data[game_data["team"] == "PHI"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,next_result
28,240.0,34.0,83.0,0.410,7.0,22.0,0.318,20.0,23.0,0.870,...,42.9,239.0,107.0,BOS,112,1,2016,2015-10-28,False,False
52,240.0,19.0,63.0,0.302,6.0,15.0,0.400,27.0,37.0,0.730,...,33.1,200.0,94.0,UTA,99,0,2016,2015-10-30,False,False
92,240.0,38.0,85.0,0.447,7.0,24.0,0.292,17.0,23.0,0.739,...,34.2,171.0,112.0,CLE,107,0,2016,2015-11-02,False,False
130,240.0,34.0,81.0,0.420,6.0,28.0,0.214,13.0,15.0,0.867,...,42.0,110.0,105.0,MIL,91,1,2016,2015-11-04,False,False
157,240.0,41.0,79.0,0.519,10.0,23.0,0.435,10.0,15.0,0.667,...,31.9,177.0,113.0,CLE,108,1,2016,2015-11-06,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17697,240.0,38.0,84.0,0.452,8.0,30.0,0.267,19.0,22.0,0.864,...,33.4,200.0,114.0,MIA,119,1,2022,2022-05-04,False,True
17700,240.0,32.0,67.0,0.478,16.0,33.0,0.485,19.0,22.0,0.864,...,37.1,121.0,127.0,MIA,79,0,2022,2022-05-06,True,True
17711,240.0,37.0,68.0,0.544,16.0,33.0,0.485,26.0,34.0,0.765,...,28.6,155.0,129.0,MIA,108,0,2022,2022-05-08,True,False
17716,240.0,31.0,85.0,0.365,9.0,32.0,0.281,14.0,15.0,0.933,...,34.6,284.0,102.0,MIA,120,1,2022,2022-05-10,False,False


In [4]:
#Handling missing values in the target column
game_data["next_result"].fillna(2, inplace=True)

C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\749701167.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  game_data["next_result"].fillna(2, inplace=True)


In [5]:
# Converting the next_result column to integers
game_data["next_result"] = game_data["next_result"].astype(int, errors="ignore")

In [6]:
# Check how many games each team won
game_data["won"].value_counts()

won
False    8886
True     8886
Name: count, dtype: int64

In [7]:
# Identifying columns with missing values
missing_values = pd.isnull(game_data)
missing_sums = missing_values.sum()
columns_with_nulls = missing_sums[missing_sums > 0]
columns_with_nulls

+/-             17772
mp_max          17772
mp_max.1        17772
+/-_opp         17772
mp_max_opp      17772
mp_max_opp.1    17772
dtype: int64

In [8]:
# Filter out invalid columns
valid_cols = game_data.columns[~game_data.columns.isin(columns_with_nulls.index)]

In [9]:
valid_cols

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'next_result'],
      dtype='object', length=142)

In [10]:
# Create a copy of the cleaned dataset
cleaned_game_data = game_data[valid_cols].copy()
cleaned_game_data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,next_result
0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False,0
1,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False,1
2,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True,1
3,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,...,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True,1
4,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,0.737,...,36.3,133.0,112.0,GSW,107,0,2022,2022-06-10,False,0
17768,240.0,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,0.867,...,94.4,300.0,112.0,BOS,94,0,2022,2022-06-13,True,1
17769,240.0,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,0.677,...,36.2,222.0,107.0,GSW,104,1,2022,2022-06-13,False,0
17770,240.0,34.0,80.0,0.425,11.0,28.0,0.393,11.0,12.0,0.917,...,31.5,186.0,111.0,GSW,103,0,2022,2022-06-16,False,2


In [11]:
# Import necessary libraries for model training
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1)
tscv = TimeSeriesSplit(n_splits=3)
feature_selector = SequentialFeatureSelector(ridge_clf, n_features_to_select=30, direction="forward", cv=tscv)

In [12]:
# Columns to be removed for model training
excluded_columns = ["season", "date", "won", "next_result", "team", "team_opp"]

In [13]:
# Selecting the relevant columns
predictor_columns = cleaned_game_data.columns[~cleaned_game_data.columns.isin(excluded_columns)]

In [14]:
# Scale the feature columns using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cleaned_game_data[predictor_columns] = scaler.fit_transform(cleaned_game_data[predictor_columns])

In [15]:
# Fit the SequentialFeatureSelector
feature_selector.fit(cleaned_game_data[predictor_columns], cleaned_game_data["next_result"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [16]:
# Get the selected predictors
selected_predictors = list(predictor_columns[feature_selector.get_support()])
selected_predictors

['mp',
 'fg%',
 '3p%',
 'orb',
 'ts%',
 'usg%',
 '3p%_max',
 'ft_max',
 'fta_max',
 '+/-_max',
 'drb%_max',
 'trb%_max',
 'tov%_max',
 'usg%_max',
 'mp_opp',
 'fg_opp',
 '3p_opp',
 'ft%_opp',
 'blk_opp',
 'usg%_opp',
 'fga_max_opp',
 '3p_max_opp',
 'ft_max_opp',
 'ft%_max_opp',
 'blk_max_opp',
 'pf_max_opp',
 'pts_max_opp',
 'drb%_max_opp',
 'blk%_max_opp',
 'usg%_max_opp']

In [17]:
# Function for backtesting the model
def evaluate_model(data_set, model, predictors_list, start=2, step=1):
    all_predictions = []

    seasons_list = sorted(data_set["season"].unique())

    for season_index in range(start, len(seasons_list), step):
        current_season = seasons_list[season_index]

        train_set = data_set[data_set["season"] < current_season]
        test_set = data_set[data_set["season"] == current_season]

        model.fit(train_set[predictors_list], train_set["next_result"])
        predictions = model.predict(test_set[predictors_list])
        prediction_series = pd.Series(predictions, index=test_set.index)

        results_combined = pd.concat([test_set["next_result"], prediction_series], axis=1)
        results_combined.columns = ["actual", "predicted"]

        all_predictions.append(results_combined)
    return pd.concat(all_predictions)

In [18]:
# Perform backtesting
model_predictions = evaluate_model(cleaned_game_data, ridge_clf, selected_predictors)

In [19]:
model_predictions

,actual,predicted
5250,1,1
5251,1,1
5252,0,0
5253,1,0
5254,0,1
...,...,...
17767,0,0
17768,1,1
17769,0,1
17770,2,1


In [20]:
# Calculate the accuracy of the predictions
from sklearn.metrics import accuracy_score

accuracy_score(model_predictions["actual"], model_predictions["predicted"])

0.5471969333972209

In [21]:
# Calculate home team win percentage
cleaned_game_data.groupby("home").apply(lambda group: group[group["won"] == 1].shape[0] / group.shape[0])

C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\1922128550.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cleaned_game_data.groupby("home").apply(lambda group: group[group["won"] == 1].shape[0] / group.shape[0])


home
0.0    0.428314
1.0    0.571686
dtype: float64

In [22]:
# Prepare data for rolling averages
rolling_data = cleaned_game_data[list(predictor_columns) + ["won", "team", "season"]]
rolling_data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,0.079,0.679245,0.277279,0.554502,0.317647,0.451923,1.0,False,NOP,2016
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,0.140,0.509434,0.160462,0.345972,0.317647,0.317308,1.0,False,CLE,2016
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,0.185,0.270440,0.088575,0.232227,0.329412,0.298077,0.0,True,CHI,2016
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,0.063,0.344864,0.215661,0.530806,0.505882,0.298077,0.0,True,GSW,2016
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,0.047,0.300839,0.019255,0.203791,0.317647,0.403846,0.0,False,ATL,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.103,0.401468,0.182285,0.208531,0.411765,0.413462,0.0,False,BOS,2022
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.124,0.423480,0.928113,1.000000,0.411765,0.288462,0.0,True,GSW,2022
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.076,0.300839,0.181001,0.630332,0.352941,0.384615,1.0,False,BOS,2022
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.063,0.300839,0.120668,0.459716,0.400000,0.375000,0.0,False,BOS,2022


In [23]:
# Function to calculate rolling averages over the last 5 games
def compute_rolling_averages(team_df):
    numeric_features = team_df.select_dtypes(include=['number'])
    rolling_averages = numeric_features.rolling(5).mean()
    return team_df.loc[:, team_df.columns.difference(numeric_features.columns)].join(rolling_averages)

# Apply the rolling averages to each team and season
rolling_data = rolling_data.groupby(["team", "season"], group_keys=False).apply(compute_rolling_averages)

C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\3314384825.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rolling_data = rolling_data.groupby(["team", "season"], group_keys=False).apply(compute_rolling_averages)


In [24]:
rolling_data

,team,won,mp,fg,fga,fg%,3p,3pa,3p%,ft,...,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,season
0,NOP,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CLE,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CHI,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GSW,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ATL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,BOS,False,0.0,0.409091,0.364706,0.397608,0.517241,0.493939,0.481473,0.330233,...,0.442202,0.0548,0.1608,0.505660,0.200513,0.497630,0.501176,0.380769,0.6,2022.0
17768,GSW,True,0.0,0.454545,0.385294,0.439713,0.503448,0.560606,0.421853,0.269767,...,0.490367,0.0428,0.1432,0.344654,0.292940,0.791469,0.487059,0.375000,0.4,2022.0
17769,BOS,False,0.0,0.390909,0.335294,0.398086,0.517241,0.493939,0.481473,0.339535,...,0.534404,0.0610,0.1490,0.491614,0.221566,0.591469,0.491765,0.396154,0.6,2022.0
17770,BOS,False,0.0,0.350000,0.320588,0.359330,0.448276,0.454545,0.453207,0.330233,...,0.386927,0.0682,0.1356,0.495807,0.160719,0.495735,0.430588,0.386538,0.4,2022.0


In [25]:
# Rename columns to reflect the rolling averages
rolling_col_names = [f"{col}_5" for col in rolling_data.columns]
rolling_data.columns = rolling_col_names

# Concatenate the rolling averages back into the main dataset
game_data_with_rolling = pd.concat([cleaned_game_data, rolling_data], axis=1)

In [26]:
game_data_with_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast%_max_opp_5,stl%_max_opp_5,blk%_max_opp_5,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,season_5
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.442202,0.0548,0.1608,0.505660,0.200513,0.497630,0.501176,0.380769,0.6,2022.0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.490367,0.0428,0.1432,0.344654,0.292940,0.791469,0.487059,0.375000,0.4,2022.0
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.534404,0.0610,0.1490,0.491614,0.221566,0.591469,0.491765,0.396154,0.6,2022.0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.386927,0.0682,0.1356,0.495807,0.160719,0.495735,0.430588,0.386538,0.4,2022.0


In [27]:
# Drop rows with missing values
game_data_with_rolling.dropna(inplace=True)

In [28]:
game_data_with_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast%_max_opp_5,stl%_max_opp_5,blk%_max_opp_5,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,season_5
107,0.0,0.568182,0.441176,0.526316,0.482759,0.393939,0.554632,0.046512,0.111111,0.270712,...,0.258486,0.0446,0.0970,0.249686,0.148652,0.243602,0.378824,0.365385,0.6,2016.0
109,0.0,0.477273,0.338235,0.509569,0.103448,0.242424,0.178147,0.441860,0.380952,0.766628,...,0.260092,0.0772,0.0674,0.433333,0.088575,0.488152,0.338824,0.361538,0.6,2016.0
117,0.0,0.409091,0.441176,0.339713,0.241379,0.257576,0.395487,0.372093,0.333333,0.735123,...,0.256422,0.0446,0.0746,0.338574,0.099615,0.408531,0.383529,0.317308,0.6,2016.0
118,0.0,0.318182,0.352941,0.296651,0.275862,0.303030,0.395487,0.255814,0.222222,0.766628,...,0.343349,0.0606,0.0566,0.558071,0.142490,0.287204,0.444706,0.363462,0.6,2016.0
119,0.0,0.318182,0.338235,0.308612,0.241379,0.348485,0.307601,0.511628,0.492063,0.672112,...,0.477982,0.0598,0.0972,0.379245,0.179461,0.326066,0.449412,0.251923,0.6,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.442202,0.0548,0.1608,0.505660,0.200513,0.497630,0.501176,0.380769,0.6,2022.0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.490367,0.0428,0.1432,0.344654,0.292940,0.791469,0.487059,0.375000,0.4,2022.0
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.534404,0.0610,0.1490,0.491614,0.221566,0.591469,0.491765,0.396154,0.6,2022.0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.386927,0.0682,0.1356,0.495807,0.160719,0.495735,0.430588,0.386538,0.4,2022.0


In [29]:
# Create helper functions to shift columns
def shift_column(team_df, col_name):
    return team_df[col_name].shift(-1)

def add_shifted_column(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_column(x, col_name))

# Add shifted columns to the dataset
game_data_with_rolling["home_shifted"] = add_shifted_column(game_data_with_rolling, "home")
game_data_with_rolling["team_opp_shifted"] = add_shifted_column(game_data_with_rolling, "team_opp")
game_data_with_rolling["date_shifted"] = add_shifted_column(game_data_with_rolling, "date")

C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\1945389229.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_column(x, col_name))
C:\Users\tuand\AppData\Local\Temp\ipykernel_27608\1945389229.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_column(x, col_name))
C:\Users\t

In [30]:
game_data_with_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,season_5,home_shifted,team_opp_shifted,date_shifted
107,0.0,0.568182,0.441176,0.526316,0.482759,0.393939,0.554632,0.046512,0.111111,0.270712,...,0.249686,0.148652,0.243602,0.378824,0.365385,0.6,2016.0,1.0,OKC,2015-11-05
109,0.0,0.477273,0.338235,0.509569,0.103448,0.242424,0.178147,0.441860,0.380952,0.766628,...,0.433333,0.088575,0.488152,0.338824,0.361538,0.6,2016.0,0.0,POR,2015-11-05
117,0.0,0.409091,0.441176,0.339713,0.241379,0.257576,0.395487,0.372093,0.333333,0.735123,...,0.338574,0.099615,0.408531,0.383529,0.317308,0.6,2016.0,1.0,BRK,2015-11-04
118,0.0,0.318182,0.352941,0.296651,0.275862,0.303030,0.395487,0.255814,0.222222,0.766628,...,0.558071,0.142490,0.287204,0.444706,0.363462,0.6,2016.0,1.0,MIL,2015-11-06
119,0.0,0.318182,0.338235,0.308612,0.241379,0.348485,0.307601,0.511628,0.492063,0.672112,...,0.379245,0.179461,0.326066,0.449412,0.251923,0.6,2016.0,1.0,PHI,2015-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.505660,0.200513,0.497630,0.501176,0.380769,0.6,2022.0,0.0,GSW,2022-06-13
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.344654,0.292940,0.791469,0.487059,0.375000,0.4,2022.0,0.0,BOS,2022-06-16
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.491614,0.221566,0.591469,0.491765,0.396154,0.6,2022.0,1.0,GSW,2022-06-16
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.495807,0.160719,0.495735,0.430588,0.386538,0.4,2022.0,NaN,None,None


In [31]:
# Merge with opponent team's last 5 games
final_data = game_data_with_rolling.merge(
    game_data_with_rolling[rolling_col_names + ["team_opp_shifted", "date_shifted", "team"]],
    left_on=["team", "date_shifted"],
    right_on=["team_opp_shifted", "date_shifted"]
)
final_data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_5_y,tov%_max_opp_5_y,usg%_max_opp_5_y,ortg_max_opp_5_y,drtg_max_opp_5_y,total_opp_5_y,home_opp_5_y,season_5_y,team_opp_shifted_y,team_y
0,0.0,0.568182,0.441176,0.526316,0.482759,0.393939,0.554632,0.046512,0.111111,0.270712,...,0.0974,0.278616,0.113222,0.363981,0.496471,0.438462,0.4,2016.0,CHI,OKC
1,0.0,0.477273,0.338235,0.509569,0.103448,0.242424,0.178147,0.441860,0.380952,0.766628,...,0.1390,0.334382,0.215661,0.392417,0.416471,0.342308,0.6,2016.0,MEM,POR
2,0.0,0.318182,0.352941,0.296651,0.275862,0.303030,0.395487,0.255814,0.222222,0.766628,...,0.0816,0.442767,0.169961,0.486256,0.428235,0.401923,0.4,2016.0,NYK,MIL
3,0.0,0.454545,0.323529,0.495215,0.275862,0.287879,0.413302,0.255814,0.206349,0.833139,...,0.0970,0.249686,0.148652,0.243602,0.378824,0.365385,0.6,2016.0,OKC,CHI
4,0.0,0.318182,0.279412,0.356459,0.172414,0.151515,0.423990,0.720930,0.603175,0.791132,...,0.0626,0.296436,0.161489,0.325118,0.390588,0.407692,0.6,2016.0,TOR,ORL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,0.0,0.545455,0.426471,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,...,0.1390,0.459329,0.258793,0.762085,0.560000,0.448077,0.4,2022.0,BOS,GSW
16848,0.0,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.1608,0.505660,0.200513,0.497630,0.501176,0.380769,0.6,2022.0,GSW,BOS
16849,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.1298,0.459958,0.232349,0.719431,0.557647,0.405769,0.4,2022.0,BOS,GSW
16850,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.1490,0.491614,0.221566,0.591469,0.491765,0.396154,0.6,2022.0,GSW,BOS


In [32]:
# Check merge output
final_data[["team_x", "team_opp_shifted_x", "team_y", "team_opp_shifted_y", "date_shifted"]]

,team_x,team_opp_shifted_x,team_y,team_opp_shifted_y,date_shifted
0,CHI,OKC,OKC,CHI,2015-11-05
1,MEM,POR,POR,MEM,2015-11-05
2,NYK,MIL,MIL,NYK,2015-11-06
3,OKC,CHI,CHI,OKC,2015-11-05
4,TOR,ORL,ORL,TOR,2015-11-06
...,...,...,...,...,...
16847,BOS,GSW,GSW,BOS,2022-06-10
16848,GSW,BOS,BOS,GSW,2022-06-13
16849,BOS,GSW,GSW,BOS,2022-06-13
16850,GSW,BOS,BOS,GSW,2022-06-16


In [33]:
# Remove irrelevant columns
columns_to_exclude = list(final_data.columns[final_data.dtypes == "object"]) + excluded_columns
columns_to_exclude

['team_x',
 'team_opp',
 'date',
 'team_5_x',
 'team_opp_shifted_x',
 'date_shifted',
 'team_5_y',
 'team_opp_shifted_y',
 'team_y',
 'season',
 'date',
 'won',
 'next_result',
 'team',
 'team_opp']

In [34]:
# Select remaining predictor columns
final_predictor_columns = final_data.columns[~final_data.columns.isin(columns_to_exclude)]
final_predictor_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'ast%_max_opp_5_y', 'stl%_max_opp_5_y', 'blk%_max_opp_5_y',
       'tov%_max_opp_5_y', 'usg%_max_opp_5_y', 'ortg_max_opp_5_y',
       'drtg_max_opp_5_y', 'total_opp_5_y', 'home_opp_5_y', 'season_5_y'],
      dtype='object', length=413)

In [35]:
# Re-fit the SequentialFeatureSelector on the new dataset
feature_selector.fit(final_data[final_predictor_columns], final_data["next_result"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [36]:
# Get the selected predictors
final_selected_predictors = list(final_predictor_columns[feature_selector.get_support()])
final_selected_predictors

['fga',
 'usg%',
 'orb%_max',
 'trb%_max',
 'usg%_opp',
 'ft%_max_opp',
 'trb%_max_opp',
 'ortg_max_opp',
 '3p%_5_x',
 'usg%_5_x',
 'ft_max_5_x',
 'trb_max_5_x',
 '+/-_max_5_x',
 'orb%_max_5_x',
 'ast%_opp_5_x',
 'usg%_opp_5_x',
 '3p_max_opp_5_x',
 '3p%_max_opp_5_x',
 'trb_max_opp_5_x',
 'stl_max_opp_5_x',
 'pf_max_opp_5_x',
 'home_shifted',
 'ft_5_y',
 'usg%_5_y',
 'ast_max_5_y',
 '+/-_max_5_y',
 'usg%_opp_5_y',
 'trb_max_opp_5_y',
 'stl_max_opp_5_y',
 'orb%_max_opp_5_y']

In [37]:
# Backtest the model on the final dataset
final_predictions = evaluate_model(final_data, ridge_clf, final_selected_predictors)
final_predictions

,actual,predicted
4993,1,0
4994,1,1
4995,1,1
4996,0,0
4997,1,1
...,...,...
16847,0,1
16848,1,1
16849,0,0
16850,1,0


In [38]:
# Calculate the final accuracy
accuracy_score(final_predictions["actual"], final_predictions["predicted"])

0.6173370435955814

In [39]:
# accuracy increases from 0.54 to 0.62